In [2]:
import os
from io import BytesIO
from pathlib import Path

from docling.document_converter import DocumentConverter
from docling.backends.pdf import PyPdfiumDocumentBackend
from docling.pipelines import StandardPdfPipeline
from docling.pipelines.config import PdfFormatOption, PdfPipelineOptions
from docling.protos.base_pb2 import DocumentStream, InputFormat
from docling.protos.pdf_options_pb2 import TableFormerMode

# Giả sử bạn có hàm get_object để lấy file từ S3 hoặc một nguồn lưu trữ khác
# from my_s3_utils import get_object

def doc_converter():
    """
    Tạo một document converter với các tùy chọn được tùy chỉnh.
    """
    # Tải các model cần thiết từ Hugging Face về máy, ví dụ như model cho việc nhận dạng bảng.
    # Việc này chỉ cần làm một lần.
    tmp_path = StandardPdfPipeline.download_models_hf(local_dir=Path("/tmp/docling_models"))

    # Bắt đầu cấu hình pipeline cho file PDF
    pdf_options = PdfPipelineOptions(artifacts_path=tmp_path)

    # Tùy chỉnh các lựa chọn:
    pdf_options.do_ocr = False  # Giả sử đây là file PDF gốc (native), không phải file scan.
    pdf_options.do_table_structure = True  # BẬT tính năng nhận dạng cấu trúc bảng.
    pdf_options.table_structure_options.do_cell_matching = True # Bật tính năng khớp ô trong bảng.
    pdf_options.ocr_options.use_gpu = False # Không dùng GPU cho OCR (vì đã tắt OCR)

    # Đây là tùy chọn quan trọng nhất!
    # Sử dụng chế độ 'ACCURATE' của TableFormer để có độ chính xác cao nhất khi nhận dạng bảng.
    pdf_options.table_structure_options.mode = TableFormerMode.ACCURATE

    doc_converter = (
        DocumentConverter(
            # Áp dụng các tùy chọn đã cấu hình cho định dạng PDF
            format_options={
                InputFormat.PDF: PdfFormatOption(
                    pipeline_options=pdf_options,
                    backend=PyPdfiumDocumentBackend, # Sử dụng backend PyPdfium
                ),
            },
        )
    )
    return doc_converter

# --- Luồng xử lý chính ---

# 1. Khởi tạo converter đã được tùy chỉnh
converter = doc_converter()

# 2. Lấy file PDF từ một nguồn (ví dụ: S3)
# source_bucket = "my-bucket"
# file_key = "reports/financial_report.pdf"
# body = get_object(bucket=source_bucket, key=file_key)
# file_name = os.path.basename(file_key)

# Để demo, ta đọc file từ local
file_path = "private-test-input\\Public001.pdf" # Thay bằng đường dẫn file PDF của bạn
file_name = os.path.basename(file_path)
with open(file_path, "rb") as f:
    body_stream = BytesIO(f.read())

# 3. Chuẩn bị đầu vào cho converter
source = DocumentStream(name=file_name, stream=body_stream)

# 4. Thực hiện chuyển đổi
result = converter.convert(source=source)

# 5. Xuất kết quả ra định dạng Markdown
md_text = result.document.export_to_markdown()

# 6. In kết quả
print(md_text)

d:\Utilities\miniconda\envs\t\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'docling.backends'

In [6]:
!pip show docling

Name: docling
Version: 2.57.0
Summary: SDK and CLI for parsing PDF, DOCX, HTML, and more, to a unified document representation for powering downstream workflows such as gen AI applications.
Home-page: https://github.com/docling-project/docling
Author: 
Author-email: Christoph Auer <cau@zurich.ibm.com>, Michele Dolfi <dol@zurich.ibm.com>, Maxim Lysak <mly@zurich.ibm.com>, Nikos Livathinos <nli@zurich.ibm.com>, Ahmed Nassar <ahn@zurich.ibm.com>, Panos Vagenas <pva@zurich.ibm.com>, Peter Staar <taa@zurich.ibm.com>
License-Expression: MIT
Location: D:\Utilities\miniconda\envs\t\Lib\site-packages
Requires: accelerate, beautifulsoup4, certifi, docling-core, docling-ibm-models, docling-parse, filetype, huggingface_hub, lxml, marko, openpyxl, pandas, pillow, pluggy, polyfactory, pydantic, pydantic-settings, pylatexenc, pypdfium2, python-docx, python-pptx, rapidocr, requests, rtree, scipy, tqdm, typer
Required-by: 


In [5]:
!python -c "import docling; print(docling.__version__); print(dir(docling))"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
AttributeError: module 'docling' has no attribute '__version__'
